## Tensorflow Operations

This notebook contains 4 main parts. 

- constants in tensorflow
- variables in tensorflow
- placeholders in tensorflow
- Saving and restoring graphs

##### Overview 

Tensorflow has two main objects that are used when building code. The graph and the session. If your code can be naturally separated in those two parts then probably tensorflow is a good fit. What are graphs and sessions though?

A graph is a set of nodes and edges. In tensorflow graphs define computations but do not compute anything. Graphs don't hold values (except for constants), they simply define what to do with the data. 

- Example: A Linear regression can be represented as a graph. There is a variable `w` containing the weights of each of the features and a bias variable `b`. Both `w` and `b` are naturally defined as `tf.variable` objects. Inside the graph it is also natural to define operations to change the variables of the graph, such as the weights and bias (using the gradient of the mean squared error for example). This operations are defined in the graph but they are not executed now (this is done inside the session, explained below).

Once a computation, or a model, is defined we might want to execute it (or do the computation). This is when sessions are nedded. A session is paired with a graph and inside the session the operations on the graph can be executed.  When the session is open with a particular graph the resources are allocated (memory for storing the variables for example). Once the session is closed this resources are freed.

- Going back to our previous example: Once the linear regression graph is defined we can pass data and execute a particular computation, for example `update_model`.


#### Let us recall about sessions and tensor operations...

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
from matplotlib  import pyplot as plt
%matplotlib inline

np.__version__, tf.__version__

('1.13.0', '1.2.0')

In [2]:
# Let us recall the different parts of a tensorflow program
# 1) A graph defining operations
# 2) A session that takes a graph and evaluates (fetches) variables, in this case, x
a = tf.constant(2)
b = tf.constant(2)
x = tf.add(a,b)
with tf.Session() as sess:
    print("value in x: ",sess.run(x))

value in x:  4


In [3]:
# inside the with tf.Session as sess statement 
# sess.run(x) is the same as x.eval() 
with tf.Session() as sess:
    print("value in x: ", x.eval())  

value in x:  4


## Tensorboard

We can visualize our session using tensorboard

In [6]:
a = tf.constant(2, name="Pedro")
b = tf.constant(2, name="Maria")
x = tf.add(a,b, name="add")

with tf.Session() as sess:
    writer = tf.summary.FileWriter("./graphs", sess.graph)
    
    print(sess.run(x))
    writer.close()

4


In the terminal you can write

    tensorboard --logdir="./graphs" --port 6006
    
Then open your browser and go to
 
    http://localhost:6006/
    
Inside the GRAPHS tab (at the top) you can see the operation graph of the add operation defined.

# Constants

### About tf.constant constants

In [ ]:
a = tf.constant([[1, 2]], name="a")
b = tf.constant([[0, 1], [2, 3]], name="b")
x = tf.add(a, b, name="add")
y = tf.multiply(a, b, name="multiply")
z = tf.matmul(a, b, name="matmul")

with tf.Session() as sess:
    x_val, y_val, z_val = sess.run([x, y, z])
    print("x val: \n",  x_val, "\ny_val: \n", y_val, "\nz_val: \n", z_val)

#### Filling tensor with zeros, ones or a specified value

Some usefull functions

- `tf.zeros(shape, dtype=tf.float32, name=None)`

- `tf.ones(shape, dtype=tf.float32, name=None)`

- `tf.zeros_like(input_tensor, dtype=None, name=None, optimize=True)`

- `tf.ones_like(input_tensor, dtype=None, name=None, optimize=True)`

- `tf.fill(dims, value, name=None)`

Example

     tf.fill([2,3],5) ===> [[5,5,5], [5,5,5]]


In [ ]:
aux = np.zeros((2,3))
aux

In [ ]:
np.zeros_like(aux)

In [ ]:
#like np.zeros
input_tensor = tf.zeros(shape=[2,3], dtype=tf.float32, name=None)

In [ ]:
# like np.zeros_like
tf.zeros_like(input_tensor, dtype=None, name=None, optimize=True)

### Ranges

- `tf.linspace`

- `tf.range`

In [ ]:
# Get `num` equally spaced points between `start` and `stop`
aux = tf.linspace(start=1., stop=10., num=5, name=None) 

with tf.Session() as sess:
    print("value in x: ", aux.eval())  

In [ ]:
aux = tf.range(start=5, limit=15, delta=2) 

with tf.Session() as sess:
    print("value in x: ", aux.eval())  

In [ ]:
aux = tf.range(20) 

with tf.Session() as sess:
    print("value in x: ", aux.eval())  

### Randomly generated constants

- `tf.random_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)`

- `tf.truncated_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)`

- `tf.random_uniform(shape, minval=0, maxval=None, dtype=tf.float32, seed=None, name=None)`

- `tf.radon_shuffle(value, seed=None, name=None)`

- `tf.random_crop(value, size, seed=None, name=None)`

- `tf.multinomial(logits, num_samples, seed=None, name=None)`

- `tf.random_gamma(shape, alpha, beta=None, dtype=tf.float32, seed=None, name=None)`


##### Reproducible results

- `tf.set_random_seed(seed)`


In [ ]:
tf.reset_default_graph()
x = tf.random_normal(shape=(10,2), mean=0.0, stddev=1.0,seed=123)
with tf.Session() as sess:
    print("value in x: ", x.eval())  

In [ ]:
tf.reset_default_graph()
x = tf.random_normal(shape=(10,2), mean=0.0, stddev=1.0,seed=123)
with tf.Session() as sess:
    print("value in x: ", x.eval())  

In [ ]:
tf.reset_default_graph()
x = tf.random_normal(shape=(10,2), mean=0.0, stddev=1.0)
with tf.Session() as sess:
    print("value in x: ", x.eval())  

### Be carefull with constants

Important:

**Only use constants for primitive types.
Use variables or readers for more data that 
requires more memory**

- Constants are stored in the graph definition

In [ ]:
tf.reset_default_graph()
import tensorflow as tf
# you will see value of my_const stored in the graph’s definition
my_const = tf.constant([1.0], name="my_const")
with tf.Session() as sess:
    print(sess.graph.as_graph_def())

In [ ]:
tf.reset_default_graph()

g2 = tf.Graph()
with g2.as_default():
    my_const = tf.constant([1.0], name="my_const")
    
print(g2.as_graph_def())

# About variables

Variables need to be initialized in a session, if you want to evaluate them. By evaluating we mean  to compute variable.eval() or sess.run(variable).

The following cell is an example where we want to evaluate `f` which depends on variables not initialized. Therefore the code will not work.

##### How to initialize a variable

To initialize a variable `x` use

    sess.run(x.initializer)
    
To initialize a variable `x` in a `with` statement

    with tf.Session() as sess:
        x.initializer.run()

##### How to initialize a subset of variables

    init_ab = tf.variables_initializer([a, b], name="init_ab")
    with tf.Session() as sess:
        sess.run(init_ab)

##### How to initialize all variables in a session

To initialize all variables in a session use

    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)


In [ ]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x * y + y + 1

# This won't work, since variables have not been initialized in the session
sess = tf.Session()
result = sess.run(f)
print(result)

If we initialize the variables the code will work as expected

In [ ]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x * y + y + 1

# This won't work, since variables have not been initialized in the session
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

In [ ]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x * y + y + 1

with tf.Session() as sess:
    sess.run(x.initializer)
    sess.run(y.initializer)
    result = sess.run(f)
    print(result)
    
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = sess.run(f)
    print(result)

Initializing all variables with `tf.global_variables_initializer()`

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    result = sess.run(f)
    print(result)

#### Evaluating a variable

Recall that `var.eval()` is the same as `sess.run(W)`

In [ ]:
W = tf.Variable(tf.truncated_normal([700, 10]))
with tf.Session() as sess:
    sess.run(W.initializer)
    print(W.eval())

In [ ]:
W = tf.Variable(tf.truncated_normal([700, 10]))
with tf.Session() as sess:
    sess.run(W.initializer)
    print(sess.run(W))

#### tf.Variable.assing()

This function allows to initialize and assign a value to a variable.



In [ ]:
W = tf.Variable(3.14)
W.assign(10)
with tf.Session() as sess:
    sess.run(W.initializer)   # Will print 3.14 since its the initial variable def 
    print(W.eval())

In [ ]:
W = tf.Variable(3.14)
assign_op = W.assign(10)
with tf.Session() as sess:
    sess.run(assign_op)   # running the assign operation we wrote 10 inside variable W
    print(W.eval())

In [ ]:
# create a variable whose original value is 2
my_var = tf.Variable(2, name="my_var") 
# assign a * 2 to a and call that op a_times_two
my_var_times_two = my_var.assign(2 * my_var)

with tf.Session() as sess:
    sess.run(my_var.initializer)
    print(sess.run(my_var_times_two))

In [ ]:
# create a variable whose original value is 2
my_var = tf.Variable(2, name="my_var") 
# assign a * 2 to a and call that op a_times_two
my_var_times_two = my_var.assign(2 * my_var)

with tf.Session() as sess:
    sess.run(my_var.initializer)
    print(sess.run(my_var_times_two)) # >> 4
    print(sess.run(my_var_times_two)) # >> 8
    print(sess.run(my_var_times_two)) # >> 16

#### Session vs InteractiveSession

You sometimes see InteractiveSession instead of Session

The only difference is an InteractiveSession makes itself the default 

In [ ]:
sess = tf.InteractiveSession()
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b
# We can just use 'c.eval()' without specifying the context 'sess'
print(c.eval())
sess.close()

# Placeholders

```python
    tf.placeholder(dtype, shape=None, name=None)
```

#### Some notes on placeholders and variables

- tf.Variable needs an initial value when you declare it.
- tf.Variable values are saved in the graph of a session (and require initial values)
- tf.placeholder doesn't need an initial value, it is specified at run time with the feed_dict argument inside Session.run, the values of a placeholder are never saved.

#### Why the distinction

It comes naturally from the context in which tensorflow was build. Variables will usually be the parameters of our models. We can (and only need to) save or restore the Variables to save or rebuild the graph.

Placeholders are mostly holders for the different datasets (for example training data or test data) but Variables are trained in the training process and remain the same (to predict the outcome of the input or map the inputs and outputs[labels] of the samples) later until you retrain the model(using different or the same samples to fill into the Placeholders often through the dict, for instance session.run(a_graph, dict={a_placeholder_name: sample_values}), Placeholders are also passed as parameters to set models).

To sum up, if the values are from the samples (observations you already have) you safely make a placeholder to hold them, while if you need a parameter to be trained harness a Variable (simply put, set the Variables for the values you want to get using TF automatically). 



In [ ]:
# create a placeholder of type float 32-bit, shape is a vector of 3 elements
a = tf.placeholder(tf.float32, shape=[3])

# create a constant of type float 32-bit, shape is a vector of 3 elements
b = tf.constant([5, 5, 5], tf.float32)

# use the placeholder as you would a constant or a variable
c = a + b  # Short for tf.add(a, b)

with tf.Session() as sess:
    # feed [1, 2, 3] to placeholder a via the dict {a: [1, 2, 3]} fetch value of c
    print(sess.run(c, {a: [1, 2, 3]}))
    print(sess.run(c, {a: [0, 1, 1]}))



You can feed_dict any feedable tensor.
Placeholder is just a way to indicate that 
something must be fed

```
tf.Graph.is_feedable(tensor) 
```

This can be helpful for debugging or testing

In [ ]:
# create operations, tensors, etc (using the default graph)
a = tf.add(2, 5)
b = tf.multiply(a, 3)

with tf.Session() as sess:
    # define a dictionary that says to replace the value of 'a' with 15
    replace_dict = {a: 15}
    # Run the session, passing in 'replace_dict' as the value to 'feed_dict'
    print(sess.run(b, feed_dict=replace_dict)) 

#### Be carefull with lazy evaluation

#### The following cell is GOOD

In [ ]:
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')
z = tf.add(x, y) # you create the node for add node before executing the graph

#graph = tf.Graph([x,y,z])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('./my_graph/l2', sess.graph)
    for _ in range(10):
        sess.run(z)
    writer.close()

#### The following cell is BAD!

In [ ]:
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('./my_graph/l2', sess.graph)
    for _ in range(10):
        sess.run(tf.add(x, y)) # someone decides to be clever to save one line of code
    writer.close()

#### Why is the second answer bad?

Both give the same value of z.

In the first case tensorflow knows beforehand that z = x+y, once the session is started the memory for z is allocated. z is evaluated 10 times but there is no need to allocate 10 times memory.

In the second case while running the session  tf.add(x,y) is evaluated 10 times and there are 10 allocations "for z".


#### How to use tensroflow properly

- Separate definition of ops from computing/running ops 
- Use Python property to ensure function is also loaded once the first time it is called

# Saving and restoring graphs

Tensorflow is build to facilitate saving models to disk, during execution (just in case your computer crashes) or at the end of the execution.

To save a graph you need to use a `Saver` node at the end of the construction phase (after all nodes are created). During execution `save()` must be called to save the model, passing as argument the session and the path of the checkpoint file.


- `tf.train.Saver()`: to save the session
     
- `saver.restore(sess, path_to_ckpt)`: to restore a session

In [ ]:
tf.reset_default_graph()
x = tf.Variable(10, name='x')
y = tf.Variable(10, name='y')
saver = tf.train.Saver({"x":x, "y":y})

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.save(sess, "./saved_tests/my_save_test_begin.cpkt")
    print(sess.run(x))
    x = x + 1
    saver.save(sess, "./saved_tests/my_save_test_final.cpkt")
    print(sess.run(x))
    y=y+23

##### Restoring with save.restore
Restoring a model is as easy as calling saver.restore()

After a session is closed, we can open a new session and use `save.restore

!!! DONT GET WHY x is 11 at my_save_test_begin since x = x+1 happens after saver.save()
is executed

In [ ]:
with tf.Session() as sess:
    s= saver.restore(sess, "./saved_tests/my_save_test_begin.cpkt")
    sess.graph
    print(sess.run(x))
    print(sess.run(y))

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "./saved_tests/my_save_test_final.cpkt")
    print(sess.run(x))

By default a Saver saves and restores all variables under their own name defined in the curresnt session. For more control, you can specify which variables to save or restore, and even what names to use.

For example the following Saver will save or restore only the x
variable under the name "x"


In [ ]:
tf.reset_default_graph()

g = tf.Graph()
with g.as_default():
    x = tf.Variable(10, name='x')
    y = tf.Variable(10, name='y')
    saver = tf.train.Saver()
    x = x + 10
    
with tf.Session(graph=g) as sess:
    # The first time we use the graph we need to initialize the variables
    sess.run(tf.global_variables_initializer())
    saver.save(sess, "./saved_tests/notebook_2_print_x.cpkt")
    print(sess.run(x))

In [ ]:
with tf.Session(graph=g) as sess:
    saver.restore(sess, "./saved_tests/notebook_2_print_x.cpkt")
    #sess.run(tf.global_variables_initializer())
    x = x + 20
    z = x + 100
    print(sess.run(x))
    print(sess.run(y))
    #saver.save(sess, "./saved_tests/notebook_2_print_x.cpkt")

We can continue working with the graph later on and if we canged previously variables (for example weights of a model) the changes will be kept.

The question is... how do we store the graph to disk?

Besides 

In [ ]:
tf.reset_default_graph()
with tf.Session(graph=g) as sess:
    saver.restore(sess, "./saved_tests/notebook_2_print_x.cpkt")
    print(sess.run(x))
    print(sess.run(y))
    print(sess.run(z))

#### About saving and restoring models 
- http://www.vishakhhegde.com/savegraphtensorflow.html

## About tf.slice

- `tf.slice` allow us to get an slice (or subset) of a variable.

We can use `tf.slice` on a placeholder

In [11]:
import tensorflow as tf
import numpy as np

ph = tf.placeholder(shape=[None,None], dtype=tf.int32)

# look the -1 in the first position
x = tf.slice(ph, [0, 0], [-1, 2])

input_ = np.array([[ 1, 2, 3, 4, 5, 6, 7, 8],
                   [11,21,31,41,51,61,71,81],
                   [11,21,31,41,51,61,71,81]])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(x, feed_dict={ph: input_}))

[[ 1  2]
 [11 21]
 [11 21]]


In [12]:
x = tf.slice(ph, [0,2], [-1,4])

input_ = np.array([[ 1, 2, 3, 4, 5, 6, 7, 8],
                   [11,21,31,41,51,61,71,81],
                   [11,21,31,41,51,61,71,81]])

with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print(sess.run(x, feed_dict={ph: input_}))

[[ 3  4  5  6]
 [31 41 51 61]
 [31 41 51 61]]


#### Modelling a heap in tensorflow 

The following example shows how to from a matrix [1,400]
1) get the last 4 elements and save then in the variable `get_last_visible_vector`
2) Get 396 elements from position 4 up to position 400 save them in `history_pop_oldest_vis_vector`

In [6]:
n_his = 400
n_vis = 4

pla_his = tf.placeholder(shape=[None,None], dtype=tf.int32)
get_last_visible_vector = tf.slice(pla_his, [0, n_his - n_vis], [-1, n_vis])
get_history_pop_oldest_vis_vector = tf.slice(pla_his, [0, n_vis], [-1, n_his - n_vis])
input_ = np.array([range(0,400)])

with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print("\nHistory with oldest vis vector out")
        print(sess.run(get_last_visible_vector, feed_dict={pla_his: input_}))
        print("\nHistory with oldest vis vector out")
        history_pop_oldest_vis_vector = sess.run(get_history_pop_oldest_vis_vector, feed_dict={pla_his: input_})
        print(sess.run(get_history_pop_oldest_vis_vector, feed_dict={pla_his: input_}))
        print("History with oldest vis vector out shape:", history_pop_oldest_vis_vector.shape)    



History with oldest vis vector out
[[396 397 398 399]]

History with oldest vis vector out
[[  4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21
   22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39
   40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57
   58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75
   76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93
   94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111
  112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129
  130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147
  148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165
  166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183
  184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201
  202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219
  220 221 222 223 22

The next cell shows how to go from $h_0 = [1,2,3,4,...,400]$ up to $h_1 = [4,5,6,...,404]$ by eliminating the first 4 elements of $h_0$ and  adding 4 elements to $h_1$. This can be usefull for creating "heaps" that push out elements and get in elements.

In [14]:
n_his = 400
n_vis = 4

tf.reset_default_graph()
var_history = tf.get_variable("var_history", 
                              dtype=np.float32, 
                              initializer=np.array([range(0,400)], "float32")) 

var_all_but_oldest_history = tf.get_variable("var_all_but_oldest_history", 
                              dtype=np.float32, shape=(1, n_his-n_vis)) 

get_history_pop_oldest_vis_vector = tf.slice(var_history, [0, n_vis], [-1, n_his - n_vis])
#var_history[0][0:n_vis] = var_history[]

with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        #print(sess.run(var_history))
        
        #var_history[0][0:n_his-n_vis] = var_history[0][n_vis:n_his] 
        #var_history.assign([0,0],100)
        sess.run(var_all_but_oldest_history.assign(var_history[0:1, n_vis:n_his]))
        sess.run(var_history[0:1, 0:n_his - n_vis].assign(var_all_but_oldest_history))
        current_vis_vec = np.array([[-99,-99,-99,-99]], dtype="float32")
        sess.run(var_history[0:1, n_his-n_vis:n_his].assign(current_vis_vec))
        
        #print(sess.run(var_all_but_oldest_history))
        print(sess.run(var_history))

        #print(sess.run(get_history_pop_oldest_vis_vector ))

[[   4.    5.    6.    7.    8.    9.   10.   11.   12.   13.   14.   15.
    16.   17.   18.   19.   20.   21.   22.   23.   24.   25.   26.   27.
    28.   29.   30.   31.   32.   33.   34.   35.   36.   37.   38.   39.
    40.   41.   42.   43.   44.   45.   46.   47.   48.   49.   50.   51.
    52.   53.   54.   55.   56.   57.   58.   59.   60.   61.   62.   63.
    64.   65.   66.   67.   68.   69.   70.   71.   72.   73.   74.   75.
    76.   77.   78.   79.   80.   81.   82.   83.   84.   85.   86.   87.
    88.   89.   90.   91.   92.   93.   94.   95.   96.   97.   98.   99.
   100.  101.  102.  103.  104.  105.  106.  107.  108.  109.  110.  111.
   112.  113.  114.  115.  116.  117.  118.  119.  120.  121.  122.  123.
   124.  125.  126.  127.  128.  129.  130.  131.  132.  133.  134.  135.
   136.  137.  138.  139.  140.  141.  142.  143.  144.  145.  146.  147.
   148.  149.  150.  151.  152.  153.  154.  155.  156.  157.  158.  159.
   160.  161.  162.  163.  164.  165. 

In [16]:
n_his = 400
n_vis = 4

tf.reset_default_graph()
var_history = tf.get_variable("var_history", 
                              dtype=np.float32, 
                              initializer=np.array([range(0,400)], "float32")) 

var_all_but_oldest_history = tf.get_variable("var_all_but_oldest_history", 
                              dtype=np.float32, shape=(1, n_his-n_vis)) 

#get_history_pop_oldest_vis_vector = tf.slice(var_history, [0, n_vis], [-1, n_his - n_vis])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(10):
        sess.run(var_all_but_oldest_history.assign(var_history[0:1, n_vis:n_his]))
        sess.run(var_history[0:1, 0:n_his - n_vis].assign(var_all_but_oldest_history))
        ## current_vis_vec = "predict(current_vis_vec, his_current)"
        #current_vis_vec = np.array([[i,i,i,i]], dtype="float32")
        
        sess.run(var_history[0:1, n_his-n_vis:n_his].assign(current_vis_vec))       
    print(sess.run(var_history))


[[  40.   41.   42.   43.   44.   45.   46.   47.   48.   49.   50.   51.
    52.   53.   54.   55.   56.   57.   58.   59.   60.   61.   62.   63.
    64.   65.   66.   67.   68.   69.   70.   71.   72.   73.   74.   75.
    76.   77.   78.   79.   80.   81.   82.   83.   84.   85.   86.   87.
    88.   89.   90.   91.   92.   93.   94.   95.   96.   97.   98.   99.
   100.  101.  102.  103.  104.  105.  106.  107.  108.  109.  110.  111.
   112.  113.  114.  115.  116.  117.  118.  119.  120.  121.  122.  123.
   124.  125.  126.  127.  128.  129.  130.  131.  132.  133.  134.  135.
   136.  137.  138.  139.  140.  141.  142.  143.  144.  145.  146.  147.
   148.  149.  150.  151.  152.  153.  154.  155.  156.  157.  158.  159.
   160.  161.  162.  163.  164.  165.  166.  167.  168.  169.  170.  171.
   172.  173.  174.  175.  176.  177.  178.  179.  180.  181.  182.  183.
   184.  185.  186.  187.  188.  189.  190.  191.  192.  193.  194.  195.
   196.  197.  198.  199.  200.  201. 